In [2]:
from pprint import pprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from bs4 import BeautifulSoup as bs
import xml.etree.ElementTree as ET
%matplotlib inline

In [3]:
url_root = 'https://www.usada.org/uploads/web_hist_files/xml/'
years = list(range(2015,2019))

urls = [url_root + str(yr) + 'all.xml' for yr in years]
yr_bytes = [requests.get(u).content for u in urls]

In [4]:
print(urls)

['https://www.usada.org/uploads/web_hist_files/xml/2015all.xml', 'https://www.usada.org/uploads/web_hist_files/xml/2016all.xml', 'https://www.usada.org/uploads/web_hist_files/xml/2017all.xml', 'https://www.usada.org/uploads/web_hist_files/xml/2018all.xml']


In [5]:
for y in yr_bytes:
    print(y[:200])
    print('\n\n')

b'<TestHistory><TREC><YR>2015</YR><QTR>All</QTR><AName>Madeline Aaron</AName><NumTests>2</NumTests><Sport>Figure Skating</Sport><AFN>Madeline</AFN><ALN>Aaron</ALN></TREC><TREC><YR>2015</YR><QTR>All</QTR'



b'\xff\xfe<\x00T\x00e\x00s\x00t\x00H\x00i\x00s\x00t\x00o\x00r\x00y\x00>\x00<\x00T\x00R\x00E\x00C\x00>\x00<\x00Y\x00R\x00>\x002\x000\x001\x006\x00<\x00/\x00Y\x00R\x00>\x00<\x00Q\x00T\x00R\x00>\x00A\x00l\x00l\x00<\x00/\x00Q\x00T\x00R\x00>\x00<\x00A\x00N\x00a\x00m\x00e\x00>\x00J\x00e\x00s\x00s\x00i\x00c\x00a\x00 \x00(\x00P\x00i\x00x\x00l\x00e\x00r\x00)\x00 \x00T\x00e\x00b\x00o\x00<\x00/\x00A\x00N\x00a\x00m\x00e\x00>\x00<\x00N\x00u\x00m\x00T\x00e\x00s\x00t\x00s\x00>\x001\x00<\x00/\x00N\x00u\x00m\x00T\x00'



b'\xff\xfe<\x00T\x00e\x00s\x00t\x00H\x00i\x00s\x00t\x00o\x00r\x00y\x00>\x00<\x00T\x00R\x00E\x00C\x00>\x00<\x00Y\x00R\x00>\x002\x000\x001\x007\x00<\x00/\x00Y\x00R\x00>\x00<\x00Q\x00T\x00R\x00>\x00A\x00l\x00l\x00<\x00/\x00Q\x00T\x00R\x00>\x00<\x00A\x00N\x00a\x00m\x00e\x00>\x00M\x00a\x

In [6]:
roots = {}
athletes = {}
dfs = {}

# Create dict of xml root elements for each year. 
for i, y in enumerate(years):
    roots[y] = ET.fromstring(yr_bytes[i])
    athletes[y] = []
    
    # Filter by sport to create list of athlete dicts for each year.
    for child in roots[y]:
        player_dict = {
            'year': child[0].text,
            'sport': child[4].text,
            'fname': child[5].text,
            'lname': child[6].text,
            'tests': child[3].text
        }
        athletes[y].append(player_dict)
        
    # Create dataframe for each year
    dfs[y] = pd.DataFrame(athletes[y])
    dfs[y] = dfs[y].astype({'tests': 'int64', 'year': 'int64'})

In [7]:
for y in years:
    print(dfs[y].head(), '\n')

      fname    lname              sport  tests  year
0  Madeline    Aaron     Figure Skating      2  2015
1       Max    Aaron     Figure Skating      3  2015
2   Stephen     Abas          Wrestling      1  2015
3     Mason  Abbiate  Paralympic Soccer      1  2015
4    Jeremy   Abbott     Figure Skating      1  2015 

      fname          lname              sport  tests  year
0   Jessica  (Pixler) Tebo    Track and Field      1  2016
1  Madeline          Aaron     Figure Skating      1  2016
2       Max          Aaron     Figure Skating      2  2016
3     Mason        Abbiate  Paralympic Soccer      1  2016
4      Mara         Abbott            Cycling      8  2016 

       fname         lname                 sport  tests  year
0        Max         Aaron        Figure Skating      3  2017
1       Mara        Abbott               Cycling      5  2017
2     Taylor        Abbott              Swimming      2  2017
3  Abdihakim    Abdirahman       Track and Field     10  2017
4     Hakeem  

In [9]:
for d in dfs.values():
    print(d.loc[d.sport == 'Mixed Martial Arts', :].sort_values('tests', ascending=False).head(10), '\n')
    print(d.loc[d.sport == 'Mixed Martial Arts', :].sort_values('tests', ascending=True).head(10), '\n')

        fname      lname               sport  tests  year
2564    Ronda     Rousey  Mixed Martial Arts      8  2015
29       Jose       Aldo  Mixed Martial Arts      8  2015
1990    Conor   McGregor  Mixed Martial Arts      8  2015
1282      Dan  Henderson  Mixed Martial Arts      7  2015
1371    Holly       Holm  Mixed Martial Arts      7  2015
647   Antonio   Da Silva  Mixed Martial Arts      7  2015
2536     Yoel     Romero  Mixed Martial Arts      6  2015
2509     Luke   Rockhold  Mixed Martial Arts      6  2015
774    Rafael  Dos Anjos  Mixed Martial Arts      6  2015
184     Vitor    Belfort  Mixed Martial Arts      6  2015 

       fname      lname               sport  tests  year
24     Omari   Akhmedov  Mixed Martial Arts      1  2015
2169     Roy     Nelson  Mixed Martial Arts      1  2015
2121  Gegard    Mousasi  Mixed Martial Arts      1  2015
2083  Takeya   Mizugaki  Mixed Martial Arts      1  2015
1967  Tamdan    McCrory  Mixed Martial Arts      1  2015
1931   Jorge   Mas

In [10]:
# Filter to mma only and combine into 1 df
mma = pd.concat([df.loc[df.sport=='Mixed Martial Arts',:] for df in dfs.values()], axis=0)\
    .drop('sport', axis=1)
sports = pd.concat([df.loc[df.sport!='Mixed Martial Arts',:] for df in dfs.values()], axis=0)

In [38]:
mma['required_tests'] = np.maximum(mma['tests'] - 2, 0)
print(mma.head(10))
print(sports.head())

      fname       lname  tests  year  required_tests
24    Omari    Akhmedov      1  2015               0
29     Jose        Aldo      8  2015               6
41    Eddie     Alvarez      2  2015               0
44   Thiago       Alves      5  2015               3
50    Corey    Anderson      1  2015               0
78   Andrei    Arlovski      3  2015               1
105    Ryan       Bader      4  2015               2
106     Ali  Bagautinov      3  2015               1
131   Edson     Barboza      3  2015               1
139    Josh     Barnett      1  2015               0
      fname    lname              sport  tests  year
0  Madeline    Aaron     Figure Skating      2  2015
1       Max    Aaron     Figure Skating      3  2015
2   Stephen     Abas          Wrestling      1  2015
3     Mason  Abbiate  Paralympic Soccer      1  2015
4    Jeremy   Abbott     Figure Skating      1  2015


In [39]:
mma.groupby('year').agg([np.mean, np.sum, len])

tests            required_tests           
          mean   sum  len           mean   sum  len
year                                               
2015  2.262821   353  156       0.730769   114  156
2016  3.907850  2290  586       2.076792  1217  586
2017  4.524880  2819  623       2.653291  1653  623
2018  2.060329  1127  547       0.433272   237  547

In [40]:
sports['tests'].groupby(sports.sport).agg([np.mean, np.sum, len]).sort_values('mean', ascending=False)

,mean,sum,len
sport,,,
Professional Boxing,11.134328,746,67
Speedskating,2.901408,618,213
Tennis,2.768293,227,82
Swimming,2.641509,1960,742
Biathlon,2.596154,135,52
Triathlon,2.534759,948,374
Cycling,2.364943,2469,1044
Track and Field,2.343380,4354,1858
Bobsled and Skeleton,2.284848,377,165


In [41]:
rows = mma.shape[0]
for i in range(1,10):
    print('<={} tests: {}'.format(i, round(mma.loc[mma.tests<=i, :].shape[0]/rows, 2)))
    print('<={} required tests: {}\n'.format(i, round(mma.loc[mma.required_tests<=i, :]\
                                                      .shape[0]/rows, 2)))

<=1 tests: 0.24
<=1 required tests: 0.66

<=2 tests: 0.47
<=2 required tests: 0.77

<=3 tests: 0.66
<=3 required tests: 0.83

<=4 tests: 0.77
<=4 required tests: 0.87

<=5 tests: 0.83
<=5 required tests: 0.9

<=6 tests: 0.87
<=6 required tests: 0.93

<=7 tests: 0.9
<=7 required tests: 0.95

<=8 tests: 0.93
<=8 required tests: 0.96

<=9 tests: 0.95
<=9 required tests: 0.98

